In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2022-ai-w5p1/train.csv
/kaggle/input/2022-ai-w5p1/test.csv
/kaggle/input/2022-ai-w5p1/sample_submit.csv


In [11]:
import random
import torch
import torch.optim as optim

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [12]:
train = pd.read_csv('../input/2022-ai-w5p1/train.csv')
test = pd.read_csv('../input/2022-ai-w5p1/test.csv')
submit = pd.read_csv('../input/2022-ai-w5p1/sample_submit.csv')

In [13]:
trainX = train.drop('label', axis=1)
trainY = train['label']

In [14]:
# unique로 확인
trainY.unique()

array([5, 0, 3, 6, 4, 8, 2, 9, 1, 7])

In [15]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
trainX = scaler.fit_transform(trainX)
test = scaler.transform(test)

In [16]:
x_train = torch.FloatTensor(trainX)
y_train = torch.LongTensor(trainY)
x_test = torch.FloatTensor(test)

In [18]:
# softmax
import torch.nn.functional as F

nb_data = x_train.shape[1]
nb_class = len(trainY.unique())

W = torch.zeros([nb_data, nb_class], requires_grad=True)
b = torch.zeros(nb_class, requires_grad=True)

optimizer = optim.SGD([W,b], lr=0.1)

nb_epochs = 10000
for epoch in range(nb_epochs+1) :
    
    hypothesis = F.softmax(x_train.matmul(W)+b, dim=1)
    cost = F.cross_entropy(x_train.matmul(W)+b, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch%1000 == 0 :
        print(epoch, cost.item())

0 2.3025851249694824
100 0.345892071723938
200 0.31040191650390625
300 0.29472604393959045
400 0.28529539704322815
500 0.27876174449920654
600 0.2738455832004547
700 0.26994165778160095
800 0.26672276854515076
900 0.2639945149421692
1000 0.26163315773010254


In [19]:
len(y_train.unique())

10

In [20]:
hypothesis = F.softmax(x_test.matmul(W)+b, dim=1)
predict = torch.argmax(hypothesis, dim=1)

In [21]:
submit['label'] = predict

In [22]:
submit.to_csv('submit3.csv', index=False)